In [1]:
from functools import reduce
import vyper
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from vyper.user import Model
from vyper.utils.tools import StatisticalTools as st
from vyper.user import Model
from vyper.user.explorer import DataProfiler
from vyper.utils.tools import StatisticalTools as st
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pandas.api.types import is_string_dtype
from openpyxl import Workbook
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import math
import warnings
import string

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.drawing.text import CharacterProperties
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

In [2]:
## EDITS
def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
                     variable_order, other_segment = False, file = 'Profile', exclude = '',
                     PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
    

    profile_data = pd.DataFrame(profile_data)
    
    if (segments is None):
        segments = pd.unique(profile_data[segment_var])
        other_segment = False
        segment_names = str(segments)
    if (not (isinstance(other_segment, bool))):
        raise ValueError('other must be True or False.')
    if (segment_names is None):
        segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    if (len(segments)+other_segment!=len(segment_names)):
        if(len(segments)==len(segment_names)):
            segment_names = [segment_names, 'other']
        else:
            warnings.warn('Incorrect length of names, replacing by segments')
            segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    ## output -> [1 2 0]

    varclass = pd.DataFrame()
    for col in profile_data.columns:
        varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)
        
    # Continuous Variables that were classified as categorical
    varclass[varclass.columns in continuous] = 'continuous'
    if (not (include is None)):
        exclude = varclass.columns[varclass.columns not in include]
    # Variables to exclude
    varclass[varclass.columns in exclude] = 'exclude'
    if (excludeother): 
        varclass[varclass.columns =='Categorical+other'] = 'exclude'
    continuous_var_cuts = dict()

    
    for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
        if not continuous_var_bounds:
            continuous_var_bounds = dict()
        if variable in continuous_var_bounds:
            cut_bins = continuous_var_bounds[variable]
            continuous_var_cuts[variable] = pd.cut(profile_data[variable], bins=cut_bins)
                
        else:
            continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,squash_extremes=False)

    col_order = []
    index_order = []
    ### TODO: array([2, 0, 1]) Always encode 0 as US
    ### Change the US population as BASELINE
    for seg in segments:
        profile_1 = DataProfiler(profile_data[profile_data[segment_var] == seg]).create_profile(number_of_bins = 10,
                                                                            cts_cuts = continuous_var_cuts)
        if (seg == 'Baseline'):
            profile = profile_1
            profile.columns = ["Variable", "Category",  "US_Count","US_Percent"]
            for cols in profile.columns:
                col_order.append(cols)


        else:
            profile_1.columns = ["Variable", "Category",  "Count_"+str(seg), "Percent_"+str(seg)]
            col_order.append("Count_"+str(seg))
            col_order.append("Percent_"+str(seg))
            profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
            profile['US Population vs ' + "Segment_" + str(seg)] = (profile["US_Percent"]/profile["Percent_"+str(seg)]) * 100
            index_order.append('US Population vs ' + "Segment_" + str(seg))
            
#     for i in range(len(segments)):
#         profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
#                                                                             cts_cuts = continuous_var_cuts)
#         if (i == 0):
#             profile = profile_1
#             profile.columns = ["Variable", "Category",  "US_Count","US_Percent"]
#             for cols in profile.columns:
#                 col_order.append(cols)


#         else:
#             profile_1.columns = ["Variable", "Category",  "Count_"+str(i), "Percent_"+str(i)]
#             col_order.append("Count_"+str(i))
#             col_order.append("Percent_"+str(i))
#             profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
#             profile['US Population vs ' + "Segment_" + str(i)] = (profile["US_Percent"]/profile["Percent_"+str(i)]) * 100
#             index_order.append('US Population vs ' + "Segment_" + str(i))        

    profile = profile[col_order+index_order]

    for i in profile.columns:

        if ('Count' in i):
            profile[i][profile[i].isna()] = 0
        if ('Percent' in i):
            profile[i][profile[i].isna()] = 0
            
            
    if file:
        import os
        # Make Profile ####
        filesave = file + '.xlsx'
        
        if os.path.exists(filesave):
            os.remove(filesave)
        CreateXLProfile_Snap(profile, savepath = filesave)
        
        if PPT:
            wb = Workbook()

            profile['Category'] = profile['Category'].astype(str)

            all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
            # make new sheet the active sheet we are working on
            all_var_profiling_ws = wb.active

            # remove gridlines from the sheet
            all_var_profiling_ws.sheet_view.showGridLines = False

            # set border line thickness
            thick = Side(border_style="thick", color="000000")
            # medium = Side(border_style="medium", color="000000")
            thin = Side(border_style="thin", color="000000")

            # get max rows and cols of profiling df
            max_rows = profile.shape[0]
            max_cols = profile.shape[1]
            
            input_cols = list(string.ascii_uppercase)

            j = 4
            for z in range(0, len(profile)):
                if (z != 0):
                    if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
                        # create sheet
                        all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
                        # make new sheet the active sheet we are working on
                        all_var_profiling_ws = wb.active

                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                        j = 4
                    else:
                        incol = 2
                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                            incol = incol + 1
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                            if (y % 2 == 1 and y > 2):
                                current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                                incol = incol + 1
                                current_cell.font = Font(bold=True)
                                current_cell.alignment = Alignment(horizontal='center', vertical='center')
                                current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                                current_cell.value = 'Index'
                                if y == max_cols - 1:
                                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                                else:
                                    current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
                incol = 2
                for y in range(len(profile.columns)):
                    current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                    incol = incol + 1
                    current_cell.value = profile.iloc[z, y]
                    if y == 0:
                        current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                    else:
                        current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
                    if (y % 2 == 1 and y > 2):
                        current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                        incol = incol + 1
                        current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
                        if y == max_cols - 1:
                            current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                        else:
                            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

                j = j + 1
            
            filesave = file + 'PPT.xlsx'
        
            if os.path.exists(filesave):
                os.remove(filesave)
            
            wb.save(filesave)
            
    return profile

def CreateXLProfile_Snap(profile, savepath):
    wb = Workbook()

    profile['Category'] = profile['Category'].astype(str)

    # create sheet
    all_var_profiling_ws = wb.create_sheet('profile', 0)

    # make new sheet the active sheet we are working on
    all_var_profiling_ws = wb.active

    # remove gridlines from the sheet
    all_var_profiling_ws.sheet_view.showGridLines = False

    # set border line thickness
    thick = Side(border_style="thick", color="000000")
    # medium = Side(border_style="medium", color="000000")
    thin = Side(border_style="thin", color="000000")

    # get max rows and cols of profiling df
    max_rows = profile.shape[0]
    max_cols = profile.shape[1]

    input_rows = range(4, max_rows + 4)
    input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

    y = 0
    for i in input_cols:
        x = "3"
        x = i + x

        all_var_profiling_ws[x].font = Font(bold=True)
        current_cell = all_var_profiling_ws[x]
        current_cell.alignment = Alignment(horizontal='center', vertical='center')
        current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

        if y == 0:
            current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
        elif y == max_cols - 1:
            current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
        else:
            current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

        all_var_profiling_ws[x] = profile.columns[y]
        y = y + 1
    
    print("Worked till here")
    counter = []
    counter_val = 1
    for index,i in enumerate(input_cols[2::]):
        x = "2"
        x = i + x
        if index%2==1:
            all_var_profiling_ws.merge_cells(f"{counter[-1]}:{x}")
            all_var_profiling_ws[counter[-1]].font = Font(bold=True)
            current_cell = all_var_profiling_ws[counter[-1]]
            current_cell.alignment = Alignment(horizontal='center', vertical='center')
            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
            continue


        if i==input_cols[-1]:
            break
        if i=='D':
            all_var_profiling_ws[x] = "US POPULATION"
            all_var_profiling_ws.merge_cells(start_row=2, start_column=4, end_row=2, end_column=5)
            counter.append(x)
            continue
        all_var_profiling_ws[x] = 'Segment ' + str(counter_val)
        counter_val+=1
        counter.append(x)
        # all_var_profiling_ws.merge_cells(start_row=2, start_column=5, end_row=2, end_column=5)
        # all_var_profiling_ws.merge_cells(f'{x}:D2')

    
    def style_range(ws, cell_range, border=Border(), fill=None, font=None,alignment=None):
        
        """
        Apply styles to a range of cells as if they were a single cell.

        :param ws:  Excel worksheet instance
        :param range: An excel range to style (e.g. A1:F20)
        :param border: An openpyxl Border
        :param fill: An openpyxl PatternFill or GradientFill
        :param font: An openpyxl Font object
        """

        top = Border(top=border.top)
        left = Border(left=border.left)
        right = Border(right=border.right)
        bottom = Border(bottom=border.bottom)

        first_cell = ws[cell_range.split(":")[0]]
        if alignment:
            ws.merge_cells(cell_range)
            first_cell.alignment = alignment

        rows = ws[cell_range]
        if font:
            first_cell.font = font

        for cell in rows[0]:
            cell.border = cell.border + top
        for cell in rows[-1]:
            cell.border = cell.border + bottom

        for row in rows:
            l = row[0]
            r = row[-1]
            l.border = l.border + left
            r.border = r.border + right
            if fill:
                for c in row:
                    c.fill = fill


    border = Border(top=thick, left=thick, right=thick, bottom=thick)

    for range1 in all_var_profiling_ws.merged_cells.ranges:
        style_range(all_var_profiling_ws, str(range1), border=border)

    

    y = 0
    for i in input_cols:
        z = 0
        for j in input_rows:
            all_var_profiling_ws.row_dimensions[j].height = 25
            x = str(j)
            x = i + x
            # print(x)
            current_cell = all_var_profiling_ws[x]
            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

            if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

            all_var_profiling_ws[x] = profile.iloc[z, y]

            if y == 0:
                current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
            elif y == max_cols - 1:
                current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


            z = z + 1
        y = y + 1
    

    
    if (savepath != ''):
        wb.save(savepath)

In [3]:
# Read in the demonstration Epsilon Response Model Data - Internal Append
# eps_data = pd.read_csv(r"Data/EpsilonResponseModelData_InternalAppend.csv")
eps_data = pd.read_csv('Data/selected_attributes.csv')

# Output length to confirm read in
len(eps_data)

19852

In [4]:
# Read in the Epsilon Data Dictionary we created
epsilon_dict = pd.read_csv('Data/EpsilonDataDictionary.csv')

# Pull list of Selected Attributes from Snowflake Headers
variable_list = epsilon_dict['Snowflake Headers'].tolist()
# variable_list.append('seg')

# Selected Attributes only
new_df = eps_data[variable_list]
new_df

,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,STATE,...,PROPERTY_LOT_SIZE_IN_ACRES_DUPLICATE,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK
0,1,12,34,7040,1908,470,C029,247 ELMWOOD AVE FL 2,MAPLEWOOD,NJ,...,NaN,70,11,11,NaN,8,7,196003,0,10
1,1,6,9,6870,1907,273,C001,27 EDGEWATER DR,OLD GREENWICH,CT,...,B,80,11,14,645.0,7,7,110003,0,3
2,1,7,34,7054,1937,725,C026,1480 US HIGHWAY 46 APT 57B,PARSIPPANY,NJ,...,NaN,68,4,3,NaN,5,1,417062,3,1
3,1,4,36,10954,7443,346,C019,534 AVALON GARDENS DR,NANUET,NY,...,NaN,76,11,11,NaN,2,6,113011,1,4
4,1,19,34,8807,1120,337,C025,633 CEDARBROOK RD,BRIDGEWATER,NJ,...,D,23,13,12,365.0,7,8,508022,2,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19847,1,22,34,7452,2222,202,C051,20 ABBINGTON TER,GLEN ROCK,NJ,...,A,6,13,14,NaN,1,9,222005,0,2
19848,1,7,9,6840,6132,154,C014,15 MARSHALL RIDGE RD,NEW CANAAN,CT,...,NaN,51,11,14,NaN,2,7,351005,0,11
19849,1,4,36,10021,4854,654,C224,520 E 72ND ST APT 6E,NEW YORK,NY,...,NaN,41,8,12,NaN,1,4,124007,0,1
19850,1,20,36,10591,3803,370,C016,37 LAKE AVE,TARRYTOWN,NY,...,B,23,12,13,350.0,8,8,114004,0,1


In [5]:
# Split the data in half just for testing purposes of dummy groups
cutoff = int(len(new_df)/2)
baseline_df = new_df.iloc[:cutoff,:]
baseline_df = new_df.iloc[:cutoff,:]
customer1_df = new_df.iloc[cutoff:,:]

# Create Dummy Groups
baseline_df['Group'] = 'Baseline'
customer1_df['Group'] = 'Customer_1'
             
# Merge the two dataframes together again
final_df = baseline_df.append(customer1_df)
final_df

,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,STATE,...,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK,Group
0,1,12,34,7040,1908,470,C029,247 ELMWOOD AVE FL 2,MAPLEWOOD,NJ,...,70,11,11,NaN,8,7,196003,0,10,Baseline
1,1,6,9,6870,1907,273,C001,27 EDGEWATER DR,OLD GREENWICH,CT,...,80,11,14,645.0,7,7,110003,0,3,Baseline
2,1,7,34,7054,1937,725,C026,1480 US HIGHWAY 46 APT 57B,PARSIPPANY,NJ,...,68,4,3,NaN,5,1,417062,3,1,Baseline
3,1,4,36,10954,7443,346,C019,534 AVALON GARDENS DR,NANUET,NY,...,76,11,11,NaN,2,6,113011,1,4,Baseline
4,1,19,34,8807,1120,337,C025,633 CEDARBROOK RD,BRIDGEWATER,NJ,...,23,13,12,365.0,7,8,508022,2,13,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19847,1,22,34,7452,2222,202,C051,20 ABBINGTON TER,GLEN ROCK,NJ,...,6,13,14,NaN,1,9,222005,0,2,Customer_1
19848,1,7,9,6840,6132,154,C014,15 MARSHALL RIDGE RD,NEW CANAAN,CT,...,51,11,14,NaN,2,7,351005,0,11,Customer_1
19849,1,4,36,10021,4854,654,C224,520 E 72ND ST APT 6E,NEW YORK,NY,...,41,8,12,NaN,1,4,124007,0,1,Customer_1
19850,1,20,36,10591,3803,370,C016,37 LAKE AVE,TARRYTOWN,NY,...,23,12,13,350.0,8,8,114004,0,1,Customer_1


In [6]:
len(final_df)

19852

In [7]:
# # Split the data in thres just for testing purposes of dummy groups
# cutoff = int(len(new_df)/3)
# baseline_df = new_df.iloc[:cutoff,:]
# customer1_df = new_df.iloc[cutoff:cutoff*2,:]
# customer2_df = new_df.iloc[cutoff*2:,:]             

# # # # Create Dummy Groups
# # # baseline_df['Group'] = 'Baseline'
# # # customer1_df['Group'] = 'Customer_1'
# # # customer2_df['Group'] = 'Customer_2'
             
# # # # Merge the two dataframes together again
# # # final_df = baseline_df.append(customer1_df)
# # # final_df = final_df.append(customer2_df)
# # # final_df

In [8]:
# Function to combine customer data with baseline and add the Group segment
### TODO: Figure out why the lenght of the output is wrong...seems to be appending twice or something?
def append_data(baseline_df, *customer_dfs):
    # Tag baseline group segment
    baseline_df['Group'] = "Baseline"
    append_df = baseline_df
    
    # Start a counter for group segmentation
    customer_count = 1
    
    # Loop over the customer data, adding the group segment then appending
    for customer_df in customer_dfs:
        customer_df['Group'] = f'Customer {str(customer_count)}'
        customer_count += 1
        append_df = append_df.append(customer_df)
        
    append_df.reset_index(inplace=True)
        
    return append_df

In [9]:
test_df = append_data(new_df, customer1_df)
test_df

,index,RECORD_QUALITY_CODE,NUM_SOURC_VERIFY_HH,FIPS_STATE_CODE,ZIP,ZIP4,DELIVERY_POINT_CODE,CARRIER_ROUTE,CONTRACTED_ADDRESS,POST_OFFICE_NAME,...,MT_TRADITIONAL_CABLE_PAY_TV_SERVICE_CUSTOMERS,INVESTMENT_RESOURCES_TIERS,LIQUID_RESOURCES_TIERS,MORTGAGE_LIABILITY_TIERS,SHORT_TERM_LIABILITY_TIERS,WEALTH_RESOURCES_TIERS,_2010_CENSUS_GEO_TRAC_BLOCK_GROUP,CENSUS_TRACT_SUFFIX,_2010_CENSUS_GEO_BLOCK,Group
0,0,1,12,34,7040,1908,470,C029,247 ELMWOOD AVE FL 2,MAPLEWOOD,...,70,11,11,NaN,8,7,196003,0,10,Baseline
1,1,1,6,9,6870,1907,273,C001,27 EDGEWATER DR,OLD GREENWICH,...,80,11,14,645.0,7,7,110003,0,3,Baseline
2,2,1,7,34,7054,1937,725,C026,1480 US HIGHWAY 46 APT 57B,PARSIPPANY,...,68,4,3,NaN,5,1,417062,3,1,Baseline
3,3,1,4,36,10954,7443,346,C019,534 AVALON GARDENS DR,NANUET,...,76,11,11,NaN,2,6,113011,1,4,Baseline
4,4,1,19,34,8807,1120,337,C025,633 CEDARBROOK RD,BRIDGEWATER,...,23,13,12,365.0,7,8,508022,2,13,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29773,19847,1,22,34,7452,2222,202,C051,20 ABBINGTON TER,GLEN ROCK,...,6,13,14,NaN,1,9,222005,0,2,Customer 1
29774,19848,1,7,9,6840,6132,154,C014,15 MARSHALL RIDGE RD,NEW CANAAN,...,51,11,14,NaN,2,7,351005,0,11,Customer 1
29775,19849,1,4,36,10021,4854,654,C224,520 E 72ND ST APT 6E,NEW YORK,...,41,8,12,NaN,1,4,124007,0,1,Customer 1
29776,19850,1,20,36,10591,3803,370,C016,37 LAKE AVE,TARRYTOWN,...,23,12,13,350.0,8,8,114004,0,1,Customer 1


In [10]:
len(test_df)

29778

In [11]:
# Function to read in a csv with containing the variable names and the respective bin cutoffs
def continuous_bins(filepath):
    df_continuous = pd.read_csv(filepath)
    
    continuous_bounds = {}
    for index, row in df_continuous.iterrows():
        continuous_bounds[row['Snowflake Field names']] = [float(i) for i in row['Bin'].split(sep=',')]
        
    return continuous_bounds

In [12]:
# Set up variables for snapshot
file_name = '2-seg-20k-test'
seg_var = 'Group'
dataset = final_df

# Read in file and set bins
bin_vars = continuous_bins('Data/bin_dictionary.csv')

In [13]:
# Run Snapshot Report
profile = Snapshot_Profile(profile_data = dataset, segment_var=seg_var, segments = None, segment_names = None, include = None,
                 continuous_var_bounds = bin_vars, variable_order = None, other_segment = False,
                 file = file_name, exclude = [], PPT = True, continuous = [], excludeother = False)
profile

Worked till here


,Variable,Category,US_Count,US_Percent,Count_Customer_1,Percent_Customer_1,US Population vs Segment_Customer_1
0,NUM_SOURC_VERIFY_HH,"(0.999, 5.0]",1516.0,15.273020,1558.0,15.696152,97.304236
1,NUM_SOURC_VERIFY_HH,"(5.0, 6.0]",516.0,5.198469,541.0,5.450332,95.378928
2,NUM_SOURC_VERIFY_HH,"(6.0, 9.0]",1275.0,12.845053,1273.0,12.824904,100.157109
3,NUM_SOURC_VERIFY_HH,"(9.0, 11.0]",784.0,7.898449,855.0,8.613742,91.695906
4,NUM_SOURC_VERIFY_HH,"(11.0, 13.0]",844.0,8.502922,826.0,8.321580,102.179177
...,...,...,...,...,...,...,...
6742,VEHICLE_MANUFACTURING_CODE_2,O,0.0,0.000000,3.0,0.030224,NaN
6743,VEHICLE_MILEAGE_CODE_1,P,0.0,0.000000,2.0,0.020149,NaN
6744,VEHICLE_MILEAGE_CODE_1,S,0.0,0.000000,2.0,0.020149,NaN
6745,VEHICLE_MILEAGE_CODE_1,Z,0.0,0.000000,1.0,0.010075,NaN


In [14]:
file_xlsx = file_name + '.xlsx'
CreateXLProfile_Snap(profile, file_xlsx)

Worked till here


In [15]:
# def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
#                      variable_order, other_segment = False, file = 'Profile', exclude = '',
#                      PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
#     profile_data = pd.DataFrame(profile_data)
#     if (segments is None):
#         segments = pd.unique(profile_data[segment_var])
#         other_segment = False
#         segment_names = str(segments)
#     if (not (isinstance(other_segment, bool))):
#         raise ValueError('other must be True or False.')
#     if (segment_names is None):
#         segment_names = str(segments)
#         if (other_segment):
#             segment_names = [segment_names, 'other']
#     if (len(segments)+other_segment!=len(segment_names)):
#         if(len(segments)==len(segment_names)):
#             segment_names = [segment_names, 'other']
#         else:
#             warnings.warn('Incorrect length of names, replacing by segments')
#             segment_names = str(segments)
#         if (other_segment):
#             segment_names = [segment_names, 'other']

#     varclass = pd.DataFrame()
#     for col in profile_data.columns:
#         varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)

#     # Continuous Variables that were classified as categorical
#     varclass[varclass.columns in continuous] = 'continuous'
#     if (not (include is None)):
#         exclude = varclass.columns[varclass.columns not in include]
#     # Variables to exclude
#     varclass[varclass.columns in exclude] = 'exclude'
#     if (excludeother): varclass[varclass.columns =='Categorical+other'] = 'exclude'


#     continuous_var_cuts = dict()
#     for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
#         if not continuous_var_bounds:
#             continuous_var_bounds = dict()
#         if variable in continuous_var_bounds:
#             continuous_var_cuts[variable] = continuous_var_bounds[variable]
#         else:
#             continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,
#                      squash_extremes=False)

#     for i in range(len(segments)):
#         profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
#                                                                           cts_cuts = continuous_var_cuts)
#         if (i == 0):
#             profile = profile_1
#         else:
#             profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')

#     for i in profile.columns:
#         print(i)
#         print(profile[i])
#         if ('Count' in i):
#             profile[i][profile[i].isna()] = 0
#         if ('Percent' in i):
#             profile[i][profile[i].isna()] = 0
            
            
#     if file:
#         import os
#         # Make Profile ####
#         filesave = file + '.xlsx'
        
#         if os.path.exists(filesave):
#             os.remove(filesave)
#         CreateXLProfile_Snap(profile, savepath = filesave)
        
#         if PPT:
#             wb = Workbook()

#             profile['Category'] = profile['Category'].astype(str)

#             all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
#             # make new sheet the active sheet we are working on
#             all_var_profiling_ws = wb.active

#             # remove gridlines from the sheet
#             all_var_profiling_ws.sheet_view.showGridLines = False

#             # set border line thickness
#             thick = Side(border_style="thick", color="000000")
#             # medium = Side(border_style="medium", color="000000")
#             thin = Side(border_style="thin", color="000000")

#             # get max rows and cols of profiling df
#             max_rows = profile.shape[0]
#             max_cols = profile.shape[1]
            
#             input_cols = list(string.ascii_uppercase)

#             j = 4
#             for z in range(0, len(profile)):
#                 if (z != 0):
#                     if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
#                         # create sheet
#                         all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
#                         # make new sheet the active sheet we are working on
#                         all_var_profiling_ws = wb.active

#                         for y in range(len(profile.columns)):
#                             current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
#                             current_cell.font = Font(bold=True)
#                             current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                             current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                             current_cell.value = profile.columns[y]
#                             if y == 0:
#                                 current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#                             elif y == max_cols - 1:
#                                 current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                             else:
#                                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
#                         j = 4
#                     else:
#                         incol = 2
#                         for y in range(len(profile.columns)):
#                             current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
#                             incol = incol + 1
#                             current_cell.font = Font(bold=True)
#                             current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                             current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                             current_cell.value = profile.columns[y]
#                             if y == 0:
#                                 current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#                             elif y == max_cols - 1:
#                                 current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                             else:
#                                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
#                             if (y % 2 == 1 and y > 2):
#                                 current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
#                                 incol = incol + 1
#                                 current_cell.font = Font(bold=True)
#                                 current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                                 current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                                 current_cell.value = 'Index'
#                                 if y == max_cols - 1:
#                                     current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                                 else:
#                                     current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
#                 incol = 2
#                 for y in range(len(profile.columns)):
#                     current_cell = all_var_profiling_ws.cell(row = j, column = incol)
#                     incol = incol + 1
#                     current_cell.value = profile.iloc[z, y]
#                     if y == 0:
#                         current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
#                     else:
#                         current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
#                     if (y % 2 == 1 and y > 2):
#                         current_cell = all_var_profiling_ws.cell(row = j, column = incol)
#                         incol = incol + 1
#                         current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
#                         if y == max_cols - 1:
#                             current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
#                         else:
#                             current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

#                 j = j + 1
            
#             filesave = file + 'PPT.xlsx'
        
#             if os.path.exists(filesave):
#                 os.remove(filesave)
            
#             wb.save(filesave)
            
#     return profile

In [16]:
# def CreateXLProfile_Snap(profile, savepath):
#     wb = Workbook()

#     profile['Category'] = profile['Category'].astype(str)

#     # create sheet
#     all_var_profiling_ws = wb.create_sheet('profile', 0)

#     # make new sheet the active sheet we are working on
#     all_var_profiling_ws = wb.active

#     # remove gridlines from the sheet
#     all_var_profiling_ws.sheet_view.showGridLines = False

#     # set border line thickness
#     thick = Side(border_style="thick", color="000000")
#     # medium = Side(border_style="medium", color="000000")
#     thin = Side(border_style="thin", color="000000")

#     # get max rows and cols of profiling df
#     max_rows = profile.shape[0]
#     max_cols = profile.shape[1]

#     input_rows = range(4, max_rows + 4)
#     input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

#     y = 0
#     for i in input_cols:
#         x = "3"
#         x = i + x

#         all_var_profiling_ws[x].font = Font(bold=True)
#         current_cell = all_var_profiling_ws[x]
#         current_cell.alignment = Alignment(horizontal='center', vertical='center')
#         current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

#         if y == 0:
#             current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#         elif y == max_cols - 1:
#             current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#         else:
#             current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

#         all_var_profiling_ws[x] = profile.columns[y]
#         y = y + 1

#     y = 0
#     for i in input_cols:
#         z = 0
#         for j in input_rows:
#             all_var_profiling_ws.row_dimensions[j].height = 25
#             x = str(j)
#             x = i + x
#             # print(x)
#             current_cell = all_var_profiling_ws[x]
#             current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

#             if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

#             all_var_profiling_ws[x] = profile.iloc[z, y]

#             if y == 0:
#                 current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
#                 if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                     current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#             elif y == max_cols - 1:
#                 current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
#                 if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                     current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


#             z = z + 1
#         y = y + 1
    

    
#     if (savepath != ''):
#         wb.save(savepath)